second version of training that uses aggregation mlp

In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, NodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive_io.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                     eager = True,
                     create_inverse_triples = True
                     )


In [5]:
model_name = 'nodepiece_transductive_2_io'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200
num_tokens = 20

In [8]:
model = NodePiece(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim,
        aggregation = 'mlp'
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/16.4k [00:00<?, ?it/s]

No symbolic computation of output shape.


Number of parameters: 244000
Space occupied: 976000 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_transductive_2_io created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 3.6799250841140747
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.005963533482639381
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.01755183891658351
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.011757686199611447
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0012589216930791736
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.002155778929591179
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0017073502531275153
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.000922126528602185
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0018271908140348842
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0013746586713185348
Metric: validation.head.optimistic.hits_at_1 = 0.0036006228104320747
Metric: validation.tail.optimistic.hits_at_1 = 0.014499805371739977
Metric: validation.both.optimistic.hits_at_1 

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 1.83016437292099
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.004261248940524208
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.014323274980833729
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.009292261960678968
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0016198439989238977
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0021234038285911083
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.001871624030172825
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0013222138496528765
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0018941944198283915
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0016082041347406342
Metric: validation.head.optimistic.hits_at_1 = 0.001557026080186843
Metric: validation.tail.optimistic.hits_at_1 = 0.011385753211366291
Metric: validation.both.optimistic.hits_at_1 =

...

Step: 123
Metric: loss = 0.350862517952919
Step: 123
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.5051631364276904
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.027683895831510492
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.26642351612960047
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.004819544963538647
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.011686496436595917
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.008253021165728569
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0034484647171070587
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.008667586933783989
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.006058025825445524
Metric: validation.head.optimistic.hits_at_1 = 0.4970805760996497
Metric: validation.tail.optimistic.hits_at_1 = 0.01732191514207863
Metric: validation.both.optimistic.hits_at_1 = 0.25

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 124
Metric: loss = 0.34538787603378296
Step: 124
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.5074159619286043
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.02717777587862535
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.2672968689036148
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.004842078313231468
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.012231136672198772
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.008536607958376408
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0034429915815345296
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.009030307045208815
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.006236649313371672
Metric: validation.head.optimistic.hits_at_1 = 0.49892954456987154
Metric: validation.tail.optimistic.hits_at_1 = 0.016543402101985207
Metric: validation.both.optimistic.hits_at_1 = 0.

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 125
Metric: loss = 0.3523175120353699
Step: 125
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.5069162340730576
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.030728174864623217
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.2688222044688404
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00467821815982461
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.012713382020592691
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.008695799857378006
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.003380603467680018
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.009536374037644943
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00645848875266248
Metric: validation.head.optimistic.hits_at_1 = 0.4992214869599066
Metric: validation.tail.optimistic.hits_at_1 = 0.017127286882055275
Metric: validation.both.optimistic.hits_at_1 = 0.2581

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    3.679925
1    1.830164
2    1.013080
3    0.887554
4    0.780793
..        ...
120  0.350532
121  0.351196
122  0.350863
123  0.345388
124  0.352318

[125 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/41.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.598041   0.003223     0.002385
hits_at_1                     0.594277   0.000145     0.000145
hits_at_3                     0.596276   0.000723     0.000626
hits_at_5                     0.599142   0.001397     0.001277
hits_at_10                    0.603406   0.003830     0.002650
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.033119   0.011494     0.009019
hits_at_1                     0.021365   0.002264     0.002264
hits_at_3                     0.027821   0.008334     0.004480
hits_at_5                     0.043260   0.011369     0.008936
hits_at_10                    0.053497   0.016885     0.014693
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.315580   0.007358     0.005702
hits_at_1                     0.307821   0.001204     0.001204
hits_at_3                     0.312048  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/10.3k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.504592   0.004139     0.003064
hits_at_1                     0.499513   0.000097     0.000097
hits_at_3                     0.502336   0.001265     0.000973
hits_at_5                     0.506617   0.002336     0.002336
hits_at_10                    0.511872   0.004282     0.003114
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.028841   0.013562     0.011110
hits_at_1                     0.015376   0.004087     0.004087
hits_at_3                     0.024620   0.013429     0.005644
hits_at_5                     0.043305   0.015181     0.014013
hits_at_10                    0.053231   0.020825     0.019366
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.266716   0.008851     0.007087
hits_at_1                     0.257445   0.002092     0.002092
hits_at_3                     0.263478  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/13.1k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.502886   0.003877     0.002858
hits_at_1                     0.498012   0.000076     0.000076
hits_at_3                     0.500841   0.000841     0.000612
hits_at_5                     0.504970   0.002065     0.001759
hits_at_10                    0.509711   0.005047     0.002906
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.027764   0.013557     0.011206
hits_at_1                     0.014758   0.004358     0.004358
hits_at_3                     0.023551   0.013075     0.006041
hits_at_5                     0.041214   0.015063     0.013534
hits_at_10                    0.052225   0.021334     0.019040
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.265325   0.008717     0.007032
hits_at_1                     0.256385   0.002217     0.002217
hits_at_3                     0.262196  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['aggregation'] = 'mlp'

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                      num_tokens   
17                   embedding_dim   
18                   learning_rate   
19                       optimizer   
20                      num_epochs   
21                        patience   
22                     aggregation   

                                                value  
0                              

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)